# Support Vector Machines
Support Vector Machines (SVMs) are models for regression and classification tasks. SVM models have two particularly desirable features: robustness in the presence of noisy data and applicability to a variety of data configurations. At its core, a linear SVM model is a hyperplane separating two distinct classes of data (in the case of classification problems), in such a way that the distance between the hyperplane and the nearest training data point (called the margin) is maximized. Vectors that lie on this margin are called support vectors. With the support vectors fixed, perturbations of vectors beyond the margin will not affect the model; this contributes to the model’s robustness. By substituting a kernel function for the usual inner product, one can approximate a large variety of decision boundaries in addition to linear hyperplanes.

In [14]:
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [15]:
# Greenplum Database 5.4.0 on GCP (demo machine)
%sql postgresql://gpadmin@35.184.253.255:5432/madlib
        
# PostgreSQL local
#%sql postgresql://fmcquillan@localhost:5432/madlib

# Greenplum Database 4.3.10.0
#%sql postgresql://gpdbchina@10.194.10.68:61000/madlib

u'Connected: gpadmin@madlib'

In [16]:
%sql select madlib.version();

1 rows affected.


version
"MADlib version: 1.15-dev, git revision: rc/1.14-rc1-25-gda13eb7, cmake configuration time: Tue Jul 10 21:37:52 UTC 2018, build type: release, build system: Linux-2.6.32-696.20.1.el6.x86_64, C compiler: gcc 4.4.7, C++ compiler: g++ 4.4.7"


# Classification
# 1. Create input data set

In [17]:
%%sql 
DROP TABLE IF EXISTS houses;

CREATE TABLE houses (id INT, tax INT, bedroom INT, bath FLOAT, price INT,
            size INT, lot INT);

INSERT INTO houses VALUES   
  (1 ,  590 ,       2 ,    1 ,  50000 ,  770 , 22100),
  (2 , 1050 ,       3 ,    2 ,  85000 , 1410 , 12000),
  (3 ,   20 ,       3 ,    1 ,  22500 , 1060 ,  3500),
  (4 ,  870 ,       2 ,    2 ,  90000 , 1300 , 17500),
  (5 , 1320 ,       3 ,    2 , 133000 , 1500 , 30000),
  (6 , 1350 ,       2 ,    1 ,  90500 ,  820 , 25700),
  (7 , 2790 ,       3 ,  2.5 , 260000 , 2130 , 25000),
  (8 ,  680 ,       2 ,    1 , 142500 , 1170 , 22000),
  (9 , 1840 ,       3 ,    2 , 160000 , 1500 , 19000),
 (10 , 3680 ,       4 ,    2 , 240000 , 2790 , 20000),
 (11 , 1660 ,       3 ,    1 ,  87000 , 1030 , 17500),
 (12 , 1620 ,       3 ,    2 , 118600 , 1250 , 20000),
 (13 , 3100 ,       3 ,    2 , 140000 , 1760 , 38000),
 (14 , 2070 ,       2 ,    3 , 148000 , 1550 , 14000),
 (15 ,  650 ,       3 ,  1.5 ,  65000 , 1450 , 12000);
    
SELECT * FROM houses ORDER BY id;

Done.
Done.
15 rows affected.
15 rows affected.


id,tax,bedroom,bath,price,size,lot
1,590,2,1.0,50000,770,22100
2,1050,3,2.0,85000,1410,12000
3,20,3,1.0,22500,1060,3500
4,870,2,2.0,90000,1300,17500
5,1320,3,2.0,133000,1500,30000
6,1350,2,1.0,90500,820,25700
7,2790,3,2.5,260000,2130,25000
8,680,2,1.0,142500,1170,22000
9,1840,3,2.0,160000,1500,19000
10,3680,4,2.0,240000,2790,20000



# 2. Train linear classification model
Categorical variable is price < $100,0000.

In [18]:
%%sql
DROP TABLE IF EXISTS houses_svm, houses_svm_summary;

SELECT madlib.svm_classification('houses',
                                 'houses_svm',
                                 'price < 100000',
                                 'ARRAY[1, tax, bath, size]'
                           );
SELECT * FROM houses_svm;

Done.
1 rows affected.
1 rows affected.


coef,loss,norm_of_gradient,num_iterations,num_rows_processed,num_rows_skipped,dep_var_mapping
"[0.124749754442359, -0.002823869432027, 0.0751780666986316, 0.00163774992345709]",0.647742474881,4412.03185101,100,15,0,"[False, True]"


# 3. Predict using linear model
We want to predict if house price is less than $100,000. We use the training data set for prediction as well, which is not usual but serves to show the syntax. The predicted results are in the "prediction" column and the actual data is in the "actual" column.

In [19]:
%%sql
DROP TABLE IF EXISTS houses_pred;

SELECT madlib.svm_predict('houses_svm', 
                          'houses', 
                          'id', 
                          'houses_pred');

SELECT *, price < 100000 AS actual FROM houses JOIN houses_pred USING (id) ORDER BY id;

Done.
1 rows affected.
15 rows affected.


id,tax,bedroom,bath,price,size,lot,prediction,decision_function,actual
1,590,2,1.0,50000,770,22100,False,-0.205087702693,True
2,1050,3,2.0,85000,1410,12000,False,-0.380729623714,True
3,20,3,1.0,22500,1060,3500,True,1.87946535136,True
4,870,2,2.0,90000,1300,17500,False,-0.0525856175296,True
5,1320,3,2.0,133000,1500,30000,False,-0.99577687725,False
6,1350,2,1.0,90500,820,25700,False,-2.26934097486,True
7,2790,3,2.5,260000,2130,25000,False,-4.0774934572,False
8,680,2,1.0,142500,1170,22000,True,0.195864017807,False
9,1840,3,2.0,160000,1500,19000,False,-2.4641889819,False
10,3680,4,2.0,240000,2790,20000,False,-5.54741133557,False


Count the miss-classifications:

In [20]:
%%sql
SELECT COUNT(*) FROM houses_pred JOIN houses USING (id) 
WHERE houses_pred.prediction != (houses.price < 100000);

1 rows affected.


count
6


# 4. Train using Gaussian kernel
Next generate a nonlinear model using a Gaussian kernel. This time we specify the initial step size and maximum number of iterations to run. As part of the kernel parameter, we choose 10 as the dimension of the space where we train SVM. A larger number will lead to a more powerful model but run the risk of overfitting. As a result, the model will be a 10 dimensional vector, instead of 4 as in the case of linear model.

In [21]:
%%sql
DROP TABLE IF EXISTS houses_svm_gaussian, houses_svm_gaussian_summary, houses_svm_gaussian_random;

SELECT madlib.svm_classification( 'houses',
                                  'houses_svm_gaussian',
                                  'price < 100000',
                                  'ARRAY[1, tax, bath, size]',
                                  'gaussian',
                                  'n_components=10',
                                  '',
                                  'init_stepsize=1, max_iter=200'
                           );

SELECT * FROM houses_svm_gaussian;

Done.
1 rows affected.
1 rows affected.


coef,loss,norm_of_gradient,num_iterations,num_rows_processed,num_rows_skipped,dep_var_mapping
"[-1.67275666209207, 1.5191640881642, -0.503066422926726, 1.33250956564454, 2.23009854231314, -0.0602475029497933, 1.97466397155921, 2.3668779833279, 0.577739846910355, 2.81255996089823]",0.0571869097341,1.18281830047,177,15,0,"[False, True]"


# 5. Predict using Gaussian model
The predicted results are in the "prediction" column and the actual data is in the "actual" column.

In [54]:
%%sql
DROP TABLE IF EXISTS houses_pred_gaussian;

SELECT madlib.svm_predict('houses_svm_gaussian', 
                          'houses', 
                          'id', 
                          'houses_pred_gaussian');

SELECT *, price < 100000 AS actual FROM houses JOIN houses_pred_gaussian USING (id) ORDER BY id;

Done.
1 rows affected.
15 rows affected.


id,tax,bedroom,bath,price,size,lot,prediction,decision_function,actual
1,590,2,1.0,50000,770,22100,True,1.64923454025,True
2,1050,3,2.0,85000,1410,12000,True,1.34505433447,True
3,20,3,1.0,22500,1060,3500,True,1.00000000092,True
4,870,2,2.0,90000,1300,17500,True,1.00000000712,True
5,1320,3,2.0,133000,1500,30000,False,-1.00000001729,False
6,1350,2,1.0,90500,820,25700,True,1.11113745879,True
7,2790,3,2.5,260000,2130,25000,False,-0.29148279088,False
8,680,2,1.0,142500,1170,22000,False,-1.00000000609,False
9,1840,3,2.0,160000,1500,19000,False,-1.23665846847,False
10,3680,4,2.0,240000,2790,20000,False,-1.0938201061,False


Count the miss classifications.  Note this produces a more accurate result than the linear case for this small data set:

In [55]:
%%sql
SELECT COUNT(*) FROM houses_pred_gaussian JOIN houses USING (id) 
WHERE houses_pred_gaussian.prediction != (houses.price < 100000);

1 rows affected.


count
0


# 6.   Balancing data sets
In the case of an unbalanced class-size dataset, use the 'balanced' parameter to classify when building the model:

In [56]:
%%sql
DROP TABLE IF EXISTS houses_svm_gaussian, houses_svm_gaussian_summary, houses_svm_gaussian_random;

SELECT madlib.svm_classification( 'houses',
                                  'houses_svm_gaussian',
                                  'price < 150000',
                                  'ARRAY[1, tax, bath, size]',
                                  'gaussian',
                                  'n_components=10',
                                  '',
                                  'init_stepsize=1, max_iter=200, class_weight=balanced'
                           );

SELECT * FROM houses_svm_gaussian;

Done.
1 rows affected.
1 rows affected.


coef,loss,norm_of_gradient,num_iterations,num_rows_processed,num_rows_skipped,dep_var_mapping
"[0.891926151039837, 0.169282494673541, -2.26539133689874, 0.526518499596676, -0.900664505989526, 0.508112011288015, -0.355474591147659, 1.23127975981665, 1.53694964239487, 1.46496058633682]",0.569002744458,0.989597662459,183,15,0,"[False, True]"


# Regression
# 1. Create input data set
For regression we use part of the well known abalone data set https://archive.ics.uci.edu/ml/datasets/abalone :

In [5]:
%%sql
DROP TABLE IF EXISTS abalone;

CREATE TABLE abalone (id INT, sex TEXT, length FLOAT, diameter FLOAT, height FLOAT, rings INT);

INSERT INTO abalone VALUES
(1,'M',0.455,0.365,0.095,15),
(2,'M',0.35,0.265,0.09,7),
(3,'F',0.53,0.42,0.135,9),
(4,'M',0.44,0.365,0.125,10),
(5,'I',0.33,0.255,0.08,7),
(6,'I',0.425,0.3,0.095,8),
(7,'F',0.53,0.415,0.15,20),
(8,'F',0.545,0.425,0.125,16),
(9,'M',0.475,0.37,0.125,9),
(10,'F',0.55,0.44,0.15,19),
(11,'F',0.525,0.38,0.14,14),
(12,'M',0.43,0.35,0.11,10),
(13,'M',0.49,0.38,0.135,11),
(14,'F',0.535,0.405,0.145,10),
(15,'F',0.47,0.355,0.1,10),
(16,'M',0.5,0.4,0.13,12),
(17,'I',0.355,0.28,0.085,7),
(18,'F',0.44,0.34,0.1,10),
(19,'M',0.365,0.295,0.08,7),
(20,'M',0.45,0.32,0.1,9);

SELECT * FROM abalone ORDER BY id;

Done.
Done.
20 rows affected.
20 rows affected.


id,sex,length,diameter,height,rings
1,M,0.455,0.365,0.095,15
2,M,0.35,0.265,0.09,7
3,F,0.53,0.42,0.135,9
4,M,0.44,0.365,0.125,10
5,I,0.33,0.255,0.08,7
6,I,0.425,0.3,0.095,8
7,F,0.53,0.415,0.15,20
8,F,0.545,0.425,0.125,16
9,M,0.475,0.37,0.125,9
10,F,0.55,0.44,0.15,19


# 2. Train linear regression model

In [58]:
%%sql
DROP TABLE IF EXISTS abalone_svm_regression, abalone_svm_regression_summary;

SELECT madlib.svm_regression('abalone',
                             'abalone_svm_regression',
                             'rings',
                             'ARRAY[1, length, diameter, height]'
                           );

SELECT * FROM abalone_svm_regression;

Done.
1 rows affected.
1 rows affected.


coef,loss,norm_of_gradient,num_iterations,num_rows_processed,num_rows_skipped,dep_var_mapping
"[1.998949892503, 0.918517478913099, 0.712125856084095, 0.229379472956877]",8.29033295818,23.2251777858,100,20,0,[None]


# 3. Predict using linear model

In [59]:
%%sql
DROP TABLE IF EXISTS abalone_regr;

SELECT madlib.svm_predict('abalone_svm_regression',
                          'abalone', 
                          'id', 
                          'abalone_regr');

SELECT * FROM abalone JOIN abalone_regr USING (id) ORDER BY id;

Done.
1 rows affected.
20 rows affected.


id,sex,length,diameter,height,rings,prediction,decision_function
1,M,0.455,0.365,0.095,15,2.69859233281,2.69859233281
2,M,0.35,0.265,0.09,7,2.52978851455,2.52978851455
3,F,0.53,0.42,0.135,9,2.81582324473,2.81582324473
4,M,0.44,0.365,0.125,10,2.69169595482,2.69169595482
5,I,0.33,0.255,0.08,7,2.50200311168,2.50200311168
6,I,0.425,0.3,0.095,8,2.6247486278,2.6247486278
7,F,0.53,0.415,0.15,20,2.81570330755,2.81570330755
8,F,0.545,0.425,0.125,16,2.83086784147,2.83086784147
9,M,0.475,0.37,0.125,9,2.72740469586,2.72740469586
10,F,0.55,0.44,0.15,19,2.85187680353,2.85187680353


RMS error:

In [60]:
%%sql
SELECT SQRT(AVG((rings-prediction)*(rings-prediction))) as rms_error FROM abalone 
JOIN abalone_regr USING (id);

1 rows affected.


rms_error
9.08842725553


# 4. Train using Gaussian model

In [61]:
%%sql
DROP TABLE IF EXISTS abalone_svm_gaussian_regression, abalone_svm_gaussian_regression_summary, abalone_svm_gaussian_regression_random;

SELECT madlib.svm_regression( 'abalone',
                              'abalone_svm_gaussian_regression',
                              'rings',
                              'ARRAY[1, length, diameter, height]',
                              'gaussian',
                              'n_components=10',
                              '',
                              'init_stepsize=1, max_iter=200'
                           );

SELECT * FROM abalone_svm_gaussian_regression;

Done.
1 rows affected.
1 rows affected.


coef,loss,norm_of_gradient,num_iterations,num_rows_processed,num_rows_skipped,dep_var_mapping
"[4.49016341280977, 2.19062972461334, -2.04673653356154, 1.11216153651262, 2.83478599238881, -4.23122821845785, 4.17684533744501, -5.36892552740644, 0.775782561685621, -3.62606941016707]",2.66850539542,0.974400795364,163,20,0,[None]


# 5. Predict using Gaussian model

In [6]:
%%sql
DROP TABLE IF EXISTS abalone_gaussian_regr;

SELECT madlib.svm_predict('abalone_svm_gaussian_regression', 
                          'abalone', 
                          'id', 
                          'abalone_gaussian_regr');

SELECT * FROM abalone JOIN abalone_gaussian_regr USING (id) ORDER BY id;

Done.
1 rows affected.
20 rows affected.


id,sex,length,diameter,height,rings,prediction,decision_function
1,M,0.455,0.365,0.095,15,9.9302009808,9.9302009808
2,M,0.35,0.265,0.09,7,9.87712610207,9.87712610207
3,F,0.53,0.42,0.135,9,10.0459812729,10.0459812729
4,M,0.44,0.365,0.125,10,10.018415777,10.018415777
5,I,0.33,0.255,0.08,7,9.81382643977,9.81382643977
6,I,0.425,0.3,0.095,8,9.973725783,9.973725783
7,F,0.53,0.415,0.15,20,10.1032556038,10.1032556038
8,F,0.545,0.425,0.125,16,10.0140320794,10.0140320794
9,M,0.475,0.37,0.125,9,10.0478657373,10.0478657373
10,F,0.55,0.44,0.15,19,10.0698224494,10.0698224494


Compute the RMS error.  Note this produces a more accurate result than the linear case for this small data set:

In [63]:
%%sql
SELECT SQRT(AVG((rings-prediction)*(rings-prediction))) as rms_error FROM abalone 
JOIN abalone_gaussian_regr USING (id);

1 rows affected.


rms_error
3.84176368344


# 6. Cross validation
Let's run cross validation for different initial step sizes and lambda values:

In [7]:
%%sql
DROP TABLE IF EXISTS abalone_svm_gaussian_regression, abalone_svm_gaussian_regression_summary, 
abalone_svm_gaussian_regression_random, abalone_svm_gaussian_regression_cv;

SELECT madlib.svm_regression( 'abalone',
                              'abalone_svm_gaussian_regression',
                              'rings',
                              'ARRAY[1, length, diameter, height]',
                              'gaussian',
                              'n_components=10',
                              '',
                              'init_stepsize=[0.01,1], n_folds=3, max_iter=200, lambda=[0.01, 0.1, 0.5], validation_result=abalone_svm_gaussian_regression_cv'
                           );

SELECT * FROM abalone_svm_gaussian_regression;

Done.
1 rows affected.
1 rows affected.


coef,loss,norm_of_gradient,num_iterations,num_rows_processed,num_rows_skipped,dep_var_mapping
"[4.49016341280977, 2.19062972461334, -2.04673653356154, 1.11216153651262, 2.83478599238881, -4.23122821845785, 4.17684533744501, -5.36892552740644, 0.775782561685621, -3.62606941016707]",2.63941855054,1.07622244533,163,20,0,[None]


View the summary table showing the final model parameters are those that produced 
the lowest error in the cross validation runs:

In [65]:
%sql SELECT * FROM abalone_svm_gaussian_regression_summary;

1 rows affected.


method,version_number,source_table,model_table,dependent_varname,independent_varname,kernel_func,kernel_params,grouping_col,optim_params,reg_params,num_all_groups,num_failed_groups,total_rows_processed,total_rows_skipped
SVR,1.15-dev,abalone,abalone_svm_gaussian_regression,rings,"ARRAY[1, length, diameter, height]",gaussian,"gamma=0.25, n_components=10,random_state=1, fit_intercept=False, fit_in_memory=True",NULL,"init_stepsize=1.0, decay_factor=0.9, max_iter=200, tolerance=1e-10, epsilon=0.01, eps_table=, class_weight=","lambda=0.01, norm=l2, n_folds=3",1,0,20,0


View the values for cross validation:

In [8]:
%%sql
SELECT * FROM abalone_svm_gaussian_regression_cv;

6 rows affected.


init_stepsize,lambda,mean_score,std_dev_score
1.0,0.01,-4.06711568585,0.435966381366
1.0,0.1,-4.08068428345,0.44660797513
1.0,0.5,-4.52576046087,0.20597876382
0.01,0.01,-11.0231044189,0.739956548721
0.01,0.1,-11.0244799274,0.740029346709
0.01,0.5,-11.0305445077,0.740350338532


# 7. Predict using cross-validated Gaussian regression model:

In [9]:
%%sql
DROP TABLE IF EXISTS abalone_gaussian_regr;
SELECT madlib.svm_predict('abalone_svm_gaussian_regression', 
                          'abalone', 
                          'id', 
                          'abalone_gaussian_regr');

Done.
1 rows affected.


svm_predict
""


Compute the RMS error. Note this produces a more accurate result than the previous run with the Gaussian kernel:

In [10]:
%%sql
SELECT SQRT(AVG((rings-prediction)*(rings-prediction))) as rms_error FROM abalone 
JOIN abalone_gaussian_regr USING (id);

1 rows affected.


rms_error
3.84176368344


# Novelty detection 
# 1. Train a non-linear one-class SVM
Use a Gaussian kernel using the housing data set. Note that the dependent variable is not a parameter for one-class:

In [11]:
%%sql
DROP TABLE IF EXISTS houses_one_class_gaussian, houses_one_class_gaussian_summary, houses_one_class_gaussian_random;

SELECT madlib.svm_one_class('houses',
                            'houses_one_class_gaussian',
                            'ARRAY[1,tax,bedroom,bath,size,lot,price]',
                            'gaussian',
                            'gamma=0.5,n_components=55, random_state=3',
                            NULL,
                            'max_iter=100, init_stepsize=10,lambda=10, tolerance=0'
                            );

SELECT * FROM houses_one_class_gaussian;

Done.
1 rows affected.
1 rows affected.


coef,loss,norm_of_gradient,num_iterations,num_rows_processed,num_rows_skipped,dep_var_mapping
"[0.0207901288823711, -0.00103437489314969, 0.00407820868429805, 0.0274910360546609, 0.0105696547048294, -0.00313332466259033, -0.0216703145014011, 0.0363248037825208, -0.0211400498166549, -0.00827402232219555, 0.0265909439934851, 0.0282462482323058, -0.0407407195393746, 0.0191290942177852, -0.00313542082923064, -0.0191740603622109, 0.0143626646548982, -0.0620527674181034, -0.000319831622794402, 0.00388104709972051, 0.00248129433065678, 0.00764915273571186, 0.014492283562898, 0.0184730815984353, -0.00745840880633255, -0.0232208663374367, -0.010724056217189, 0.00541494627043399, 0.0150679846777238, 0.0204022414812525, -0.0294626167089617, -0.00399506510201406, -0.0231139983460727, 0.0242203153309423, -0.0421196963278802, 0.0112202149916885, -0.00720876723524249, 0.0213674589734111, -0.00260107056222295, -0.0130652059444514, 0.0710580616012718, 0.0519822855717347, 0.00961050532247376, 0.0390561950837254, -0.0152620688050253, 0.0100336750737295, 0.0632488712630204, -0.0549714494076944, -0.007684860916257, 0.0322104572263339, -0.00832311210931705, 0.0279669244721609, 0.0455147539995411, -0.0639670005155479, -0.00965055072583972, 0.00648588125681694]",0.944016313708,14.5271059047,100,16,-1,"[-1.0, 1.0]"


# 2. Create test data
For the novelty detection using one-class, let's create a test data set using the last 3 values from the training set plus an outlier at the end (10x price):

In [12]:
%%sql
DROP TABLE IF EXISTS houses_one_class_test;

CREATE TABLE houses_one_class_test (id INT, tax INT, bedroom INT, bath FLOAT, price INT,
            size INT, lot INT);

INSERT INTO houses_one_class_test VALUES   
 (1 , 3100 ,       3 ,    2 , 140000 , 1760 , 38000),
 (2 , 2070 ,       2 ,    3 , 148000 , 1550 , 14000),
 (3 ,  650 ,       3 ,  1.5 ,  65000 , 1450 , 12000),
 (4 ,  650 ,       3 ,  1.5 ,  650000 , 1450 , 12000);
      
SELECT * FROM houses_one_class_test ORDER BY id;

Done.
Done.
4 rows affected.
4 rows affected.


id,tax,bedroom,bath,price,size,lot
1,3100,3,2.0,140000,1760,38000
2,2070,2,3.0,148000,1550,14000
3,650,3,1.5,65000,1450,12000
4,650,3,1.5,650000,1450,12000


# 3. Predict using Gaussian one-class novelty detection model
Result shows the last row predicted to be novel:

In [13]:
%%sql 
DROP TABLE IF EXISTS houses_pred;

SELECT madlib.svm_predict('houses_one_class_gaussian', 
                          'houses_one_class_test', 
                          'id', 
                          'houses_pred');

SELECT * FROM houses_one_class_test JOIN houses_pred USING (id) ORDER BY id;

Done.
1 rows affected.
4 rows affected.


id,tax,bedroom,bath,price,size,lot,prediction,decision_function
1,3100,3,2.0,140000,1760,38000,1.0,0.0662278474212
2,2070,2,3.0,148000,1550,14000,1.0,0.092124936453
3,650,3,1.5,65000,1450,12000,1.0,0.03415206006
4,650,3,1.5,650000,1450,12000,-1.0,-0.0131918729845
